In [1]:
!pip3 install torch torchvision torchaudio


In [ ]:
https://github.com/bartosz-paternoga/Chatbot/blob/master/sample_conversations.csv
https://github.com/AnMol12499/LSTM_based_chatbot/blob/main/intents.json
https://github.com/LuisFMCuriel/Chatbot/blob/master/Main.py
https://github.com/ZithaChitra/simple_chatbot/blob/master/model.py 7777777

https://github.com/IshaDagar/CHATBOT-for-helping-people-who-are-stressed-/blob/main/train_chatbot.py
https://github.com/Enkhai/pytorch-chatbot/blob/master/models.py
https://github.com/Clemagda/AI-Chat-bot/blob/main/chatgui.py

In [2]:
!pip install torchtext

In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
df_train = pd.read_csv('train.txt', header=None, sep=';', names=['Input', 'Sentiment'], encoding='utf-8')
df_test = pd.read_csv('test.txt', header=None, sep=';', names=['Input', 'Sentiment'], encoding='utf-8')
df_val = pd.read_csv('va.txt', header=None, sep=';', names=['Input', 'Sentiment'], encoding='utf-8')


In [11]:
tokenizer = get_tokenizer("basic_english")

def gather_tokens(data_iter):
    tokens = []
    for text in data_iter:
        tokens.extend(tokenizer(text))
    return tokens

# Tüm tokenleri tek bir listede topluyoruz
all_tokens = gather_tokens(df_train['Input'])

# Bu token listesinden kelime hazinesi oluşturuyoruz
vocab = build_vocab_from_iterator([all_tokens], specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])



In [12]:
def text_pipeline(x):
    return vocab(tokenizer(x))

def pad_sequence_fn(batch):
    batch = [torch.tensor(text_pipeline(item), dtype=torch.long) for item in batch]
    return pad_sequence(batch, padding_value=vocab["<unk>"], batch_first=True)

X_train = []
for text in df_train['Input']:
    X_train.append(text_pipeline(text))

X_val = []
for text in df_val['Input']:
    X_val.append(text_pipeline(text))

X_test = []
for text in df_test['Input']:
    X_test.append(text_pipeline(text))


X_train_pad = pad_sequence_fn(df_train['Input'])
X_val_pad = pad_sequence_fn(df_val['Input'])
X_test_pad = pad_sequence_fn(df_test['Input'])

label_dict = {'joy': 0, 'anger': 1, 'love': 2, 'sadness': 3, 'fear': 4, 'surprise': 5}
Y_train = df_train['Sentiment'].replace(label_dict).values
Y_val = df_val['Sentiment'].replace(label_dict).values
Y_test = df_test['Sentiment'].replace(label_dict).values

Y_train_f = torch.eye(6)[Y_train]
Y_val_f = torch.eye(6)[Y_val]
Y_test_f = torch.eye(6)[Y_test]


In [13]:
from torch.utils.data import DataLoader, Dataset

class EmotionDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels
    
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, idx):
        return self.inputs[idx], self.labels[idx]

train_dataset = EmotionDataset(X_train_pad, Y_train_f)
val_dataset = EmotionDataset(X_val_pad, Y_val_f)
test_dataset = EmotionDataset(X_test_pad, Y_test_f)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [8]:
class EmotionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        super(EmotionModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        embedded = self.dropout(self.embedding(text))
        lstm_out, (hidden, cell) = self.lstm(embedded)
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        output = self.fc(hidden)
        return output

model = EmotionModel(vocab_size=len(vocab), embedding_dim=64, hidden_dim=80, output_dim=6, n_layers=2, bidirectional=False, dropout=0.6)


In [25]:
import torch.nn as nn

class EmotionModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout):
        nn.Module.__init__(self)
        
        # Kelime gömme (embedding) katmanı
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM katmanı
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_dim,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            dropout=dropout,
                            batch_first=True)
        
        # Tam bağlantılı (fully connected) katman
        if bidirectional:
            self.fc = nn.Linear(hidden_dim * 2, output_dim)
        else:
            self.fc = nn.Linear(hidden_dim, output_dim)
        
        # Dropout katmanı
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, text):
        # Gömme katmanı
        embedded = self.embedding(text)
        
        # Dropout
        embedded_dropout = self.dropout(embedded)
        
        # LSTM katmanı
        lstm_out, (hidden, cell) = self.lstm(embedded_dropout)
        
        # Son gizli durumu al
        if self.lstm.bidirectional:
            hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1)
        else:
            hidden = hidden[-1,:,:]
        
        # Dropout
        hidden_dropout = self.dropout(hidden)
        
        # Tam bağlantılı katman
        output = self.fc(hidden_dropout)
        
        return output

# Model oluşturma
model = EmotionModel(vocab_size=len(vocab),
                     embedding_dim=64,
                     hidden_dim=80,
                     output_dim=6,
                     n_layers=2,
                     bidirectional=True,
                     dropout=0.6)


In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
criterion = criterion.to(device)

def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0
    for batch in iterator:
        optimizer.zero_grad()
        text, labels = batch
        text = text.to(device)
        labels = labels.to(device)
        predictions = model(text)
        loss = criterion(predictions, torch.max(labels, 1)[1])
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    with torch.no_grad():
        for batch in iterator:
            text, labels = batch
            text = text.to(device)
            labels = labels.to(device)
            predictions = model(text)
            loss = criterion(predictions, torch.max(labels, 1)[1])
            epoch_loss += loss.item()
    return epoch_loss / len(iterator)

n_epochs = 15
for epoch in range(n_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss = evaluate(model, val_loader, criterion)
    print(f'Epoch {epoch+1}, Train Loss: {train_loss:.3f}, Val Loss: {val_loss:.3f}')


Epoch 1, Train Loss: 1.588, Val Loss: 1.566
Epoch 2, Train Loss: 1.553, Val Loss: 1.487
Epoch 3, Train Loss: 1.482, Val Loss: 1.354
Epoch 4, Train Loss: 1.393, Val Loss: 1.120
Epoch 5, Train Loss: 1.249, Val Loss: 0.929
Epoch 6, Train Loss: 1.123, Val Loss: 0.786
Epoch 7, Train Loss: 1.004, Val Loss: 0.657
Epoch 8, Train Loss: 0.916, Val Loss: 0.591
Epoch 9, Train Loss: 0.843, Val Loss: 0.505
Epoch 10, Train Loss: 0.757, Val Loss: 0.464
Epoch 11, Train Loss: 0.692, Val Loss: 0.400
Epoch 12, Train Loss: 0.625, Val Loss: 0.374
Epoch 13, Train Loss: 0.590, Val Loss: 0.323
Epoch 14, Train Loss: 0.540, Val Loss: 0.290
Epoch 15, Train Loss: 0.494, Val Loss: 0.268


In [33]:
test_loss = evaluate(model, test_loader, criterion)
print(f'Test Loss: {test_loss:.3f}')

def get_key(value):
    dictionary = {0: 'joy', 1: 'anger', 2: 'love', 3: 'sadness', 4: 'fear', 5: 'surprise'}
    return dictionary[value]

def predict(sentence, tokenizer, model, maxlen=80):
    model.eval()
    sentence_lst = [sentence]
    sentence_seq = [text_pipeline(sentence) for sentence in sentence_lst]
    sentence_padded = pad_sequence_fn(sentence_lst)
    sentence_tensor = torch.tensor(sentence_padded, dtype=torch.long).to(device)
    with torch.no_grad():
        prediction = model(sentence_tensor)
        predicted_class = prediction.argmax(dim=1).item()
    return get_key(predicted_class)

print(predict("The heavy rain poured down, drenching the deserted streets, casting a gloomy shadow over the city.", tokenizer, model))


Test Loss: 0.245
sadness


/var/folders/10/f6l__m_960b6hcgcw1vdk_wr0000gn/T/ipykernel_1077/3074684875.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sentence_tensor = torch.tensor(sentence_padded, dtype=torch.long).to(device)
